## 1. Dataset cleaning

Package set up

In [1]:
%pip install chars2vec
%pip install keras
%pip install tensorflow
%pip install sklearn
%pip install geopy
%pip install reverse_geocoder

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.5/148.5 kB 137.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 667.5 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 437.6 kB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 549.6 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 903.8/903.8 kB 696.1 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 507.7 kB/s eta 0:00:00a 0:00

In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
# no warning message and no exception is raised
pd.options.mode.chained_assignment = None
from utilities import basic_cleaning, restaurant_name_to_group_id, restaurant_name_to_group_id_chunk2, split_by_geo_locations, get_city_state_country

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Import dataset

In [15]:
#sep to read ctrl-a as delimiter csv
df = pd.read_csv('css_public_all_ofos_locations.csv', sep='\x01')[:1000]
df.head()

/Users/belu/.pyenv/versions/lewagon/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (0,4,5,6,8,10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,restaurant_id,name,platform,sub_platform,latitude,longitude,city,country,active,standardized_name,restaurant_chain,delivery_radius,geom
0,9744,Loco Coco,caviar,NaN,40.765004,-73.965961,NaN,US,True,loco_coco,NaN,NaN,NaN
1,4903,Musubi,caviar,NaN,45.504641,-122.643806,NaN,US,False,musubi,NaN,NaN,NaN
2,4723,The Bakers' Lounge,caviar,NaN,38.924559,-76.990115,NaN,US,False,the_bakers_lounge,NaN,NaN,NaN
3,4128,Parm - Battery Park City,caviar,NaN,40.71418,-74.015568,NaN,US,True,parm_-_battery_park_city,NaN,NaN,NaN
4,5690,Derek Test Merchant,caviar,NaN,1.0,2.0,NaN,US,False,derek_test_merchant,NaN,NaN,NaN


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 644089 entries, 0 to 644088
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   restaurant_id      644089 non-null  object 
 1   name               644086 non-null  object 
 2   platform           644087 non-null  object 
 3   sub_platform       599311 non-null  object 
 4   latitude           644086 non-null  object 
 5   longitude          644070 non-null  object 
 6   city               56196 non-null   object 
 7   country            644071 non-null  object 
 8   active             643867 non-null  object 
 9   standardized_name  643872 non-null  object 
 10  restaurant_chain   169 non-null     object 
 11  delivery_radius    354661 non-null  object 
 12  geom               0 non-null       float64
dtypes: float64(1), object(12)
memory usage: 63.9+ MB


## Data preprocessing

1) Drop duplicates based on 'name', 'platform', 'latitude', 'longitude'
<br>
2) 'Name' column: convert to lower case, trimmed white spaces and remove special characters
<br>
3) Format the 'active' column
<br>
4) Remove na in 'latitude', 'longitude', remove latitude >90

In [16]:
df = basic_cleaning(df)

original dataset shape (1000, 13)
0 null in name columns are found and removed
0 unknown in columns latitude/logitude are found and removed
3 duplicated rows are found and removed
0 unknown coordinates are found and removed
clean dataset shape (996, 13)


In [17]:
restaurant_names = df['name'].to_numpy()
restaurant_names

array(['loco coco', 'musubi', "the bakers' lounge",
       'parm - battery park city', 'derek test merchant',
       'the plant cafe - california & pine (fidi)', "minton's",
       'brazilian bowl - albany park', 'gourmet palace china bistro',
       'liberty burger - keller', "mel's deli", 'the office',
       'kizuki ramen & izakaya - crossroads', '4505 burgers and bbq',
       'cibo', 'aperto', "ziti's", 'miyako hollywood', 'da claudio',
       'doc ricketts', 'bridgetown bagels', 'coalfire pizza - west town',
       'lale', 'moloko', 'the igloo - fishtown', "capriotti's - main st",
       'kome waza - upper east side', 'palmyra',
       'sapore italiano restaurant',
       "chen's chinese & cuisine {do not use}",
       'king��s road cafe - do not enable',
       'honey butter fried chicken - oakland popup', 'lao sze chuan',
       'gyro hut - northgate', 'roebling pizza', "percy's & co.",
       'pho vinh', 'local foods kitchen', 'buredo - tenleytown',
       'barrelhouse', "drew'

# 2. Clustering to group restaurants with similar names

### Converting raw text into a numerical representation

In [6]:
import chars2vec
import keras
import tensorflow.keras

# extract features from restaurant names
c2v_model = chars2vec.load_model('eng_50')
word_features = c2v_model.vectorize_words(restaurant_names)
word_features

2022-09-29 18:02:59.908612: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


array([[ 0.15861303, -0.46890852,  0.04734099, ...,  0.23617265,
         0.97141355,  0.0083626 ],
       [ 0.34945786, -0.6346616 , -0.02211132, ...,  0.8902566 ,
        -0.07261621,  0.69557756],
       [ 0.38069007, -0.55204654, -0.9755037 , ...,  0.7209988 ,
         0.325083  , -0.70114255],
       ...,
       [-0.2531731 , -0.6657461 , -0.84740406, ...,  0.7802946 ,
         0.383949  , -0.58344376],
       [ 0.16784205, -0.72022194, -0.8319689 , ...,  0.17758211,
         0.4631939 ,  0.02438155],
       [ 0.4077866 , -0.8537185 , -0.8367111 , ...,  0.02439818,
         0.27558607, -0.6745724 ]], dtype=float32)

### Set up workers to run ML tasks on multiple workstream 

In [19]:
from dask.distributed import Client

client = Client(n_workers=6)
client

/Users/belu/.pyenv/versions/lewagon/lib/python3.8/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 57276 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:57276/status,
Dashboard: http://127.0.0.1:57276/status,Workers: 6
Total threads: 6,Total memory: 8.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:57277,Workers: 6
Dashboard: http://127.0.0.1:57276/status,Total threads: 6
Started: Just now,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:57305,Total threads: 1
Dashboard: http://127.0.0.1:57306/status,Memory: 1.33 GiB
Nanny: tcp://127.0.0.1:57283,


### Clustering

In [7]:
from sklearn.cluster import AgglomerativeClustering

clustering = AgglomerativeClustering(
    n_clusters=None,
    distance_threshold=0.3 #smaller threshold meaning more strict similarities, and more clusters
    ).fit(word_features[:500])
    
print(f'total clusters: {clustering.n_clusters_}')
clustering.distances_

total clusters: 499


array([ 0.        ,  1.16117678,  1.27294609,  1.38600077,  1.39501664,
        1.4026572 ,  1.40761975,  1.42281564,  1.43120766,  1.44036151,
        1.49049062,  1.50670381,  1.53519659,  1.5369419 ,  1.56974174,
        1.57574441,  1.59172904,  1.6018313 ,  1.60204991,  1.61175325,
        1.61184827,  1.61636836,  1.63267935,  1.64049863,  1.64960013,
        1.65121993,  1.66434461,  1.6680338 ,  1.6702232 ,  1.67933422,
        1.69020487,  1.70072438,  1.70151073,  1.71049552,  1.71384592,
        1.71536798,  1.72444894,  1.7466503 ,  1.74825594,  1.74972426,
        1.7545576 ,  1.76332687,  1.76591929,  1.76638242,  1.76774456,
        1.77225693,  1.77232284,  1.77937855,  1.7959481 ,  1.79933189,
        1.80449051,  1.81202182,  1.81891757,  1.82594654,  1.8271209 ,
        1.82860603,  1.8543326 ,  1.8573519 ,  1.86005746,  1.86515077,
        1.876443  ,  1.8767391 ,  1.87801277,  1.87900978,  1.88114878,
        1.88234647,  1.8883492 ,  1.89636616,  1.90022032,  1.90

### Append group id based on cluster results

In [18]:
restaurant_name_to_group_id(df, restaurant_names[:500], clustering)
df.shape

(996, 14)

In [21]:
restaurant_name_to_group_id_chunk2(df, restaurant_names, clustering2, n=496)
df.shape

IndexError: index 496 is out of bounds for axis 0 with size 496

# 3. Deduplicate based on lat and long

### Within cluster groups, calculate distance between each other, drop duplicates if distance less than 200m 

In [13]:
df = df.groupby('group_id').apply(split_by_geo_locations).reset_index(drop=True)
df.shape

(498, 14)

### Append city, state and country based on coordinates

In [14]:
df_ready = get_city_state_country(df)
df_ready

Loading formatted geocoded file...


,restaurant_id,name,platform,sub_platform,latitude,longitude,city,country,active,standardized_name,restaurant_chain,delivery_radius,geom,group_id,state
0,8470,black eyed pea,caviar,NaN,32.679676,-97.106031,Tarrant County,US,False,black_eyed_pea,NaN,NaN,NaN,0.0,Texas
1,67666,black eyed pea,delivery.com,NaN,-90.000000,0.000000,,GS,False,black_eyed_pea,NaN,NaN,NaN,0.0,
2,9398,donotuse,caviar,NaN,11.000000,-11.000000,Dabola,GN,False,donotuse,NaN,NaN,NaN,1.0,Faranah
3,9484,donotuse,caviar,NaN,32.780991,-96.796428,Dallas County,US,False,donotuse,NaN,NaN,NaN,1.0,Texas
4,1702,mcdonald's,delivery.com,NaN,40.777779,-73.954559,New York County,US,True,mcdonald_s,NaN,NaN,NaN,2.0,New York
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,79489,chengdu impressions,delivery.com,NaN,41.928827,-87.648839,Cook County,US,True,chengdu_impressions,NaN,NaN,NaN,487.0,Illinois
494,3572,artichoke basille's pizza - east village,caviar,NaN,40.732166,-73.983864,,US,True,artichoke_basille_s_pizza_-_east_village,NaN,NaN,NaN,488.0,New York
495,6629,hat yai,caviar,NaN,45.562839,-122.648829,Multnomah County,US,True,hat_yai,NaN,NaN,NaN,489.0,Oregon
496,6153,3 pigs bbq,caviar,NaN,37.493545,-122.246021,San Mateo County,US,True,3_pigs_bbq,NaN,NaN,NaN,490.0,California


### Save it to csv file

In [12]:
df_ready.to_csv('clean_restaurant_list_sample.csv')

### Dataset is ready! 🎉

## 4. Exploratory analysis